In [1]:
from abenc_ph_mj18 import PH_ABE, mat_math, Inner_Product
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc

import numpy as np

## Test attack

In [2]:
def ad_auth_setup(n, assump_size, ad, pks):

    pk_ad = []
    k = assump_size
    pk_ad = [[1] * k] * (k+1)

    for i in range(n-1):
        if i + 1 == ad:
            continue

        g_XTA = pks[str(i+1)]['g_1^{X^T A}']

        tmp = []
        for j in range(len(g_XTA)):
            tmp.append([x / y for x,y in zip(pk_ad[j], g_XTA[j])])
        
        pk_ad = tmp

    pk = pks[str(ad)]
    pk['g_1^{X^T A}'] = pk_ad
    
    pks[str(ad)] = pk

    return pks
    

In [3]:
def gen_omega(K, C, n, assump_size):
    H = K['g_2^h']
    k = assump_size
    C_i_s = C['C_i']

    C_ = [1] * (k + 1)
    #C_  = tmp
    for i in range(n-1):
        #print (len(C_), len(C_i_s[i]))
        C_ = [x / y for x,y in zip(C_, C_i_s[i])]

        #print (len(C_))

    return [pair(x,y) for x,y in zip(C_, H)]

In [4]:
n = 10
assump_size = 3
group = PairingGroup('MNT224')
math_lib = mat_math()

ph_abe = PH_ABE(n, assump_size, group, math_lib)

pp, msk = ph_abe.setup()
attributes = Inner_Product(group)
vec_x, vec_v = attributes.gen_x_v(n, assump_size)

pks, sks = ph_abe.auth_setup(pp)

In [5]:
ad = vec_v.index(0) + 1
ad_vec_v = [0] * (n-1) + [1]
print ('Authorized list: ', vec_v)
print ('Adversary:', ad, '\nVector belongs to AD:', ad_vec_v)
pks = ad_auth_setup(n, assump_size, ad, pks)

Authorized list:  [0, 1, 0, 0, 0, 0, 0, 0, 0, 1]
Adversary: 1 
Vector belongs to AD: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [6]:
M = group.random(GT)
print ('M: ', M)

C, vec_s = ph_abe.encrypt(pp, pks, vec_x, M)

M:  [[3112503220561093128987407945060484424440677649679706324803950921942, 14404662330266887931397377341007394082648722217863092628797461840670, 15018207371190241561263222263389105303127442989617401415622879473236], [3236108329076772331554759165989198461471186573938737283175124716830, 2651477853122033797608060410678686757655987567779434243507083077761, 12067497251952762932032005209199592462032575162257846340549568765805]]


In [7]:
K = ph_abe.keygen(pp, sks, vec_v)
print ('M_ (Usr):', ph_abe.decrypt(K, C, vec_v,pp))

M_ (Usr): [[3112503220561093128987407945060484424440677649679706324803950921942, 14404662330266887931397377341007394082648722217863092628797461840670, 15018207371190241561263222263389105303127442989617401415622879473236], [3236108329076772331554759165989198461471186573938737283175124716830, 2651477853122033797608060410678686757655987567779434243507083077761, 12067497251952762932032005209199592462032575162257846340549568765805]]


In [8]:
K_ = ph_abe.keygen(pp, sks, ad_vec_v)
omega = gen_omega(K_, C, n, assump_size)
M_ = ph_abe.decrypt(K_, C, ad_vec_v, pp)
print ('M_ (Adv):', M_ * math_lib.prod(omega))

M_ (Adv): [[3112503220561093128987407945060484424440677649679706324803950921942, 14404662330266887931397377341007394082648722217863092628797461840670, 15018207371190241561263222263389105303127442989617401415622879473236], [3236108329076772331554759165989198461471186573938737283175124716830, 2651477853122033797608060410678686757655987567779434243507083077761, 12067497251952762932032005209199592462032575162257846340549568765805]]


## Test NIZK

In [5]:
from nizk import NIZK, s_pair
from abenc_ph_mj18 import PH_ABE, mat_math, Inner_Product
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc
import time
import numpy as np

In [2]:
n = 10
assump_size = 3
group = PairingGroup('MNT224')
math_lib = mat_math()

ph_abe = PH_ABE(n, assump_size, group, math_lib)

pp, msk = ph_abe.setup()
pks, sks = ph_abe.auth_setup(pp)
nizk = NIZK(group)

In [7]:
start_time = time.time()
for i in range(10):
    s_pairs, pis = nizk.commit_pk(pp, pks[str(i+1)], sks[str(i+1)])
    check = nizk.verify_pk(pp, s_pairs, pis)
    if not check:
        print ("Check not pass")
        break
print ('time cost:', time.time() - start_time)

time cost: 1.2179341316223145


In [3]:
s_pairs, pis = nizk.commit_pk(pp, pks['1'], sks['1'])

In [4]:
nizk.verify_pk(pp, s_pairs, pis)

True

### Complex

In [1]:
import time

In [5]:
start_time = time.time()
A = pp['g_1^A'] # global pp
B = pks['1']['g_1^{X^T A}'] #pub key
s = sks['1']['X'] # secret key
R, u = nizk.commit_gA(pp, s, "abc")
nizk.verify_gA(pp, B, R, u, "abc")

R, u = nizk.commitPK(pp, pks['1'], sks['1'])
nizk.verifyPK(pp, pks['1'], R, u)

s = group.random(ZR)
e_gh = pair(group.random(G1), group.random(G2))

rps = s_pair(s, e_gh, group)
R, u = nizk.commit(rps, "abd")
nizk.verify(rps, R, u, "abd")
print ('time cost:', time.time() - start_time)



time cost: 0.1212320327758789


In [ ]:
nizk.verify_gA(pp, B, R, u, "abc")

In [10]:
A = pp['g_1^A'] # global pp
B = pks['1']['g_1^{X^T A}'] #pub key
s = sks['1']['X'] # secret key

In [11]:
R, u = nizk.commit_gA(pp, s, "abc")
nizk.verify_gA(pp, B, R, u, "abc")

True

In [12]:
R, u = nizk.commitPK(pp, pks['1'], sks['1'])
nizk.verifyPK(pp, pks['1'], R, u)

In [13]:
s = group.random(ZR)
e_gh = pair(group.random(G1), group.random(G2))

rps = s_pair(s, e_gh, group)
R, u = nizk.commit(rps, "abd")
nizk.verify(rps, R, u, "abd")

True

In [78]:
def decompose(s, A, B):
    rps = []
    # check s, A, B same dim
    s_ = np.array(s)
    A_ = np.array(A)
    B_ = np.array(B)

    if s_.shape == A_.shape == B_.shape:

        if s_.ndim == 1:
            zipped = list(zip(s_, A_, B_))
        elif s_.ndim == 2:
            zipped = [list(zip(r1, r2, r3)) for r1, r2,r3 in zip(s_,A_,B_)]

        print (zipped)
    else:
        print (s_.shape, A_.shape, B_.shape)
        print ("Not have same shape")

def same_dim(*args):
    shapes = [np.shape(np.array(arg)) for arg in args]

    return all(shape == shapes[0] for shape in shapes)

In [ ]:
s = group.random(ZR)
e_gh = pair(group.random(G1), group.random(G2))

rps = s_pair(s, e_gh, group)
R, u = nizk.commit(rps, "abd")
nizk.verify(rps, R, u, "abd")